<center><image src="https://drive.google.com/uc?id=1n3G4TdK_u6PQHcLrxB_A0HijNdigXmUH">

<h3 style="text-align: center;"><b>Школа глубокого обучения ФПМИ МФТИ, 2024</b></h3>

<h1 style="text-align: center;"><b>Домашнее задание. Библиотека sklearn и классификация с помощью KNN</b></h1>

## Описание домашнего задания

В данном задании вы будете работать с датасетом о персонажах из вселенной Игры Престолов [A Wiki of Ice and Fire](http://awoiaf.westeros.org/). Вам предстоит предсказать, кто из персонажей умрет, а кто останется вживых.



Описание данных:

* **name**: Имя персонажа

* **Title**: Социальный статус или знатность

* **House**: Дом, к которому принадлежит персонаж

* **Culture**: Социальная группа, к которой принадлежит персонаж

* **book1/2/3/4/5**: Появление персонажа в книге

* **Is noble**: Знатность персонажа, основанное на титуле

* **Age**: Отсчет времени: 305 AC

* **male**: Мужчина или женщина

* **dateOfBirth**: дата рождения

* **Spouse**: Имя супруги\а персонажа

* **Father**: Имя отца персонажа

* **Mother**: Имя матери персонажа

* **Heir**: Имя наследника персонажа

* **Is married**: Represents whether the character is married

* **Is spouse alive**: Represents whether character's spouse is alive

* **Is mother alive:** Жива ли мать персонажа

* **Is heir alive:** Жив ли наследник персонажа

* **Is father alive:** Указывает, жив ли отец персонажа

* **Number dead relations:** Количество умерших персонажей, с которыми персонаж связан

* **Popularity score:** Количество внутренних входящих и исходящих ссылок на страницу персонажей в вики http://awoiaf.westeros.org

Целевая переменная:
* **isAlive**: жив ли персонаж в книге

Оценивание:

Баллы считаются следующим образом:

1) $1.00 \geqslant score \geqslant 0.84$ --- 5 баллов

2) $0.84 > score \geqslant 0.77$ --- 4 балла

3) $0.77 > score \geqslant 0.70$ --- 3 балла

4) $0.70 > score \geqslant 0.65$ --- 2 балла

5) $0.65 > score \geqslant 0.6$ --- 1 балл

6) $0.60 > score$ --- 0 баллов

## Часть 1. Анализ и предобработка данных

Здесь вам необходимо сделать все шаги, которые обсуждались в первой части семинара.
* Предобработка данных
  * Обработка пропущенных данных
  * Создание новых признаков
  * Удаление ненужных столбцов
* Анализ данных
  * Анализ целевой переменной
  * Анализ признаков
  * Анализ влияния признаков на целевую переменную
* Подготовка данных для обучения модели

Загружаем датасет

**Задание 1.1.** Импортируйте библиотеки pandas, matplotlib, seaborn

In [263]:
#  Импортируем библиотеки pandas, matplotlib, seaborn
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np


data = pd.read_csv('game_of_thrones_train.csv', index_col = 'S.No')

In [264]:
#Index(['name', 'title', 'male', 'culture', 'dateOfBirth', 'mother', 'father',
 #      'heir', 'house', 'spouse', 'book1', 'book2', 'book3', 'book4', 'book5',
  #     'isAliveMother', 'isAliveFather', 'isAliveHeir', 'isAliveSpouse',
   #    'isMarried', 'isNoble', 'age', 'numDeadRelations', 'popularity',
    #   'isAlive'],
     # dtype='object')

drop_column_list = [ 'mother','father', 'heir',
       'isAliveMother', 'isAliveFather', 'isAliveHeir', ]
data.drop(columns=drop_column_list, inplace=True)

In [265]:
for column_name, column_data in data.items():
    na_percentage = round((column_data.isna().sum() / len(column_data)) * 100, 3)
    print(f'{column_name}: {na_percentage} % пропущенных данных')

name: 0.0 % пропущенных данных
title: 53.95 % пропущенных данных
male: 0.0 % пропущенных данных
culture: 68.658 % пропущенных данных
dateOfBirth: 82.081 % пропущенных данных
house: 24.47 % пропущенных данных
spouse: 87.155 % пропущенных данных
book1: 0.0 % пропущенных данных
book2: 0.0 % пропущенных данных
book3: 0.0 % пропущенных данных
book4: 0.0 % пропущенных данных
book5: 0.0 % пропущенных данных
isAliveSpouse: 87.155 % пропущенных данных
isMarried: 0.0 % пропущенных данных
isNoble: 0.0 % пропущенных данных
age: 82.081 % пропущенных данных
numDeadRelations: 0.0 % пропущенных данных
popularity: 0.0 % пропущенных данных
isAlive: 0.0 % пропущенных данных


In [243]:
#mean_age = data['age'][data['age'].notna()].mean()
#mean_age

# Отдельно заменим NaN на 'Rare'
data['title'] = data['title'].fillna('Rare')
data['culture'] = data['culture'].fillna('Outliers')
data['house'] = data['house'].fillna('Uncommon')
data['isAliveSpouse'] = data['isAliveSpouse'].fillna(1)
data['age'] = data['age'].fillna(35.29)
data['spouse']= data['spouse'].fillna('Unc')

# Проверим результат


# Находим количество встречаемых значений для каждого дома
house_counts = data['title'].value_counts()

# Определяем дома, которые встречаются один раз
rare_title = house_counts[house_counts <= 9].index

# Заменяем редкие дома (встречающиеся 1 раз) на 'Uncommon'
data['title'] = data['title'].replace(rare_title, 'Rare')




# Подсказка
#Упростите признак culture, объединив схожие названия в один.
cult = {
    'Summer Islands': ['summer islands', 'summer islander', 'summer isles'],
    'Ghiscari': ['ghiscari', 'ghiscaricari',  'ghis'],
    'Asshai': ["asshai'i", 'asshai'],
    'Lysene': ['lysene', 'lyseni'],
    'Andal': ['andal', 'andals'],
    'Braavosi': ['braavosi', 'braavos'],
    'Dornish': ['dornishmen', 'dorne', 'dornish'],
    'Myrish': ['myr', 'myrish', 'myrmen'],
    'Westermen': ['westermen', 'westerman', 'westerlands'],
    'Westerosi': ['westeros', 'westerosi'],
    'Stormlander': ['stormlands', 'stormlander'],
    'Norvoshi': ['norvos', 'norvoshi'],
    'Northmen': ['the north', 'northmen'],
    'Free Folk': ['wildling', 'first men', 'free folk'],
    'Qartheen': ['qartheen', 'qarth'],
    'Reach': ['the reach', 'reach', 'reachmen'],
}

# Переворачиваем словарь для обратного поиска (когда по значению можно найти ключ)
reverse_cult = {alias: key for key, aliases in cult.items() for alias in aliases}

data['culture'] = data['culture'].apply(lambda x: reverse_cult[x.lower()] if x.lower() in reverse_cult.keys() )

#sorted(data['culture'].unique())

# Находим количество встречаемых значений для каждого дома
cult_counts = data['culture'].value_counts()

# Определяем дома, которые встречаются один раз
#rare_title = cult_counts[cult_counts <= 6].index

# Заменяем редкие дома (встречающиеся 1 раз) на 'Uncommon'
#data['culture'] = data['culture'].replace(rare_title, 'Outliers')






#house_counts = data['house'].value_counts()

# Определяем дома, которые встречаются один раз
#rare_houses = house_counts[house_counts <= 12].index

# Заменяем редкие дома (встречающиеся 1 раз) на 'Uncommon'
#data['house'] = data['house'].replace(rare_houses, 'Uncommon')

#data['house'].value_counts()



# Находим количество встречаемых значений для каждого дома
house_counts = data['spouse'].value_counts()

# Определяем дома, которые встречаются один раз
rare_title = house_counts[house_counts <= 1].index

# Заменяем редкие дома (встречающиеся 1 раз) на 'Uncommon'
data['spouse'] = data['spouse'].replace(rare_title, 'Unc')




# Шаг 1: Находим среднее значение для строк, где 'age' >= 0
mean_age = data['age'][(data['age'].notna()) & (data['age'] >= 0)].mean()

# Шаг 2: Заменяем значения, которые NaN или меньше 0, на найденное среднее
data['age'] = data['age'].apply(lambda x: mean_age if pd.isna(x) or x < 0 else x)

# Шаг 3: Разделение на категории, перезаписываем столбец 'age'
bins = [0, 15, 25, 40, 65, float('inf')]  # Границы для категорий
labels = ['1', '2', '3', '4', '5']  # Метки для категорий

# Используем pd.cut для разделения на категории и перезаписываем столбец 'age'
data['age'] = pd.cut(data['age'], bins=bins, labels=labels, right=False)

# Проверяем результат





data['isPopular'] = np.where(data['popularity'] >= 0.5, 1.0, 0.0)

drop_column_list = ['popularity']
data.drop(columns=drop_column_list, inplace=True)
data['isPopular']






data['boolDeadRelations'] = np.where(data['numDeadRelations'] > 0, 1.0, 0.0)

data[['boolDeadRelations','numDeadRelations']]
drop_column_list = ['numDeadRelations']
data.drop(columns=drop_column_list, inplace=True)
data

,title,male,spouse,book1,book2,book3,book4,book5,isAliveSpouse,isMarried,isNoble,age,isAlive,isPopular,boolDeadRelations
S.No,,,,,,,,,,,,,,,
1,Rare,1,Unc,0,0,0,0,0,1.0,0,0,3,0,1.0,1.0
2,Rare,1,Unc,1,1,1,1,1,1.0,1,1,5,1,1.0,1.0
3,Ser,1,Unc,0,0,0,1,0,1.0,0,1,3,1,0.0,0.0
4,Rare,0,Viserys I Targaryen,0,0,0,0,0,0.0,1,1,2,0,0.0,0.0
5,Rare,0,Unc,0,0,0,1,0,1.0,1,1,3,1,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1553,Archmaester,1,Unc,1,0,1,1,0,1.0,0,1,3,1,0.0,0.0
1554,Rare,0,Unc,1,1,0,0,0,1.0,0,0,3,0,0.0,0.0
1555,Rare,1,Unc,0,1,1,1,1,1.0,0,0,3,0,0.0,0.0


In [244]:
for column in data.columns:
    print(f"Столбец: {column}")
    print(data[column].value_counts())
    print("\n" + "-"*40 + "\n")  # Разделитель для удобства чтения


Столбец: title
title
Rare           1141
Ser             306
Maester          29
Archmaester      21
Lord             19
Septon           16
Winterfell       15
Lady             10
Name: count, dtype: int64

----------------------------------------

Столбец: male
male
1    920
0    637
Name: count, dtype: int64

----------------------------------------

Столбец: spouse
spouse
Unc                    1535
Walder Frey               6
Craster                   4
Viserys I Targaryen       2
Kiera of Tyrosh           2
Alys Arryn                2
Stevron Frey              2
Amerei Frey               2
Rohanne Webber            2
Name: count, dtype: int64

----------------------------------------

Столбец: book1
book1
0    1341
1     216
Name: count, dtype: int64

----------------------------------------

Столбец: book2
book2
0    1047
1     510
Name: count, dtype: int64

----------------------------------------

Столбец: book3
book3
0    885
1    672
Name: count, dtype: int64

--------------

isAlive
culture_Braavosi
culture_Reach
culture_Westermen
house_Faith of the Seven
house_House Frey
house_House Greyjoy
house_House Osgrey
house_House Tyrell
title_Archmaester
title_Lady
title_Lord
title_Septon
title_Winterfell
spouse_Alys Arryn
spouse_Amerei Frey
spouse_Craster
spouse_Kiera of Tyrosh
spouse_Rohanne Webber
spouse_Stevron Frey
spouse_Viserys I Targaryen
spouse_Walder Frey

In [245]:

from sklearn.preprocessing import LabelEncoder
import pandas as pd

# Создаем копию исходных данных
df = data.copy()

# Инициализируем LabelEncoder
label_encoder = LabelEncoder()

# Для каждой категориальной переменной применим Label Encoding
categorical_columns = ['title', 'spouse', 'age']

# Применяем Label Encoding к каждой категориальной переменной
for column in categorical_columns:
    # Применяем Label Encoding к столбцу
    df[column] = label_encoder.fit_transform(df[column].astype(str))  # Преобразуем в строку для работы с NaN

# Проверяем результат
print(df.head())


      title  male  spouse  book1  book2  book3  book4  book5  isAliveSpouse  \
S.No                                                                          
1         4     1       6      0      0      0      0      0            1.0   
2         4     1       6      1      1      1      1      1            1.0   
3         6     1       6      0      0      0      1      0            1.0   
4         4     0       7      0      0      0      0      0            0.0   
5         4     0       6      0      0      0      1      0            1.0   

      isMarried  isNoble  age  isAlive  isPopular  boolDeadRelations  
S.No                                                                  
1             0        0    2        0        1.0                1.0  
2             1        1    4        1        1.0                1.0  
3             0        1    2        1        0.0                0.0  
4             1        1    1        0        0.0                0.0  
5             1     

In [246]:
for column in data.columns:
    print(f"Столбец: {column}")
    print(data[column].value_counts())
    print("\n" + "-"*40 + "\n")  # Разделитель для удобства чтения


Столбец: title
title
Rare           1141
Ser             306
Maester          29
Archmaester      21
Lord             19
Septon           16
Winterfell       15
Lady             10
Name: count, dtype: int64

----------------------------------------

Столбец: male
male
1    920
0    637
Name: count, dtype: int64

----------------------------------------

Столбец: spouse
spouse
Unc                    1535
Walder Frey               6
Craster                   4
Viserys I Targaryen       2
Kiera of Tyrosh           2
Alys Arryn                2
Stevron Frey              2
Amerei Frey               2
Rohanne Webber            2
Name: count, dtype: int64

----------------------------------------

Столбец: book1
book1
0    1341
1     216
Name: count, dtype: int64

----------------------------------------

Столбец: book2
book2
0    1047
1     510
Name: count, dtype: int64

----------------------------------------

Столбец: book3
book3
0    885
1    672
Name: count, dtype: int64

--------------

In [209]:
isAlive
culture_Braavosi
culture_Reach
culture_Westermen
house_Faith of the Seven
house_House Frey
house_House Greyjoy
house_House Osgrey
house_House Tyrell
title_Archmaester
title_Lady
title_Lord
title_Septon
title_Winterfell
spouse_Alys Arryn
spouse_Amerei Frey
spouse_Craster
spouse_Kiera of Tyrosh
spouse_Rohanne Webber
spouse_Stevron Frey
spouse_Viserys I Targaryen
spouse_Walder Frey

SyntaxError: invalid syntax (2531277625.py, line 5)

In [210]:
data1= df

In [247]:
X = df.drop(columns=['isAlive']).values
y = df['isAlive'].values
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split (X,y, test_size = 0.2 ,shuffle = True)



In [248]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_train)
# Выход pca - numpy матрица, положим ее в новую переменную со всеми фичами
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [249]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

# Шаг 1. создание модели
logistic_regression = LogisticRegression(C=1)

# Шаг 2. обучение модели
logistic_regression.fit(X_train_scaled, y_train)

# Шаг 3. Предсказание на тестовых данных
y_pred = logistic_regression.predict(X_test_scaled)



# Шаг 1. создание модели
adaboost_classifier =  AdaBoostClassifier()

# Шаг 2. обучение модели
adaboost_classifier.fit(X_train_scaled, y_train)

# Шаг 3. Предсказание на тестовых данных
y_pred = adaboost_classifier.predict(X_test_scaled)



# Шаг 1. создание модели
RandomForest_classifier =  RandomForestClassifier()

# Шаг 2. обучение модели
RandomForest_classifier.fit(X_train_scaled, y_train)

# Шаг 3. Предсказание на тестовых данных
y_pred = RandomForest_classifier.predict(X_test_scaled)





# Шаг 1. создание модели
gaussp_classifier = GaussianProcessClassifier()

# Шаг 2. обучение модели
gaussp_classifier.fit(X_train_scaled, y_train)

# Шаг 3. Предсказание на тестовых данных
y_pred = gaussp_classifier.predict(X_test_scaled)



# Шаг 1. создание модели
Gaussian_NB =  GaussianNB()

# Шаг 2. обучение модели
Gaussian_NB.fit(X_train_scaled, y_train)

# Шаг 3. Предсказание на тестовых данных
y_pred = Gaussian_NB.predict(X_test_scaled)



# Шаг 1. создание модели
KNN =  KNeighborsClassifier()

# Шаг 2. обучение модели
KNN.fit(X_train_scaled, y_train)

# Шаг 3. Предсказание на тестовых данных
y_pred = KNN.predict(X_test_scaled)




# Шаг 1. создание модели
SVC =  SVC()

# Шаг 2. обучение модели
SVC.fit(X_train_scaled, y_train)

# Шаг 3. Предсказание на тестовых данных
y_pred = SVC.predict(X_test_scaled)


# Шаг 1. создание модели
DTC =  DecisionTreeClassifier()

# Шаг 2. обучение модели
DTC.fit(X_train_scaled, y_train)

# Шаг 3. Предсказание на тестовых данных
y_pred = DTC.predict(X_test_scaled)


In [250]:
from sklearn.metrics import accuracy_score


# Шаг 3. Предсказание на тестовых данных
y_pred = logistic_regression.predict(X_test_scaled)

# Шаг 4. Оценка предсказания по метрике accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy : %.4f" % accuracy)






# Шаг 3. Предсказание на тестовых данных
y_pred = adaboost_classifier.predict(X_test_scaled)

# Шаг 4. Оценка предсказания по метрике accuracy
accuracy = accuracy_score(y_test, y_pred)

# Шаг 4. Оценка предсказания по метрике accuracy

print("Accuracy : %.4f" % accuracy)




# Шаг 3. Предсказание на тестовых данных
y_pred = RandomForest_classifier.predict(X_test_scaled)

# Шаг 4. Оценка предсказания по метрике accuracy
accuracy = accuracy_score(y_test, y_pred)

# Шаг 4. Оценка предсказания по метрике accuracy

print("Accuracy : %.4f" % accuracy)




# Шаг 3. Предсказание на тестовых данных
y_pred = gaussp_classifier.predict(X_test_scaled)

# Шаг 4. Оценка предсказания по метрике accuracy
accuracy = accuracy_score(y_test, y_pred)

# Шаг 4. Оценка предсказания по метрике accuracy

print("Accuracy : %.4f" % accuracy)





# Шаг 3. Предсказание на тестовых данных
y_pred = Gaussian_NB.predict(X_test_scaled)

# Шаг 4. Оценка предсказания по метрике accuracy
accuracy = accuracy_score(y_test, y_pred)

# Шаг 4. Оценка предсказания по метрике accuracy

print("Accuracy : %.4f" % accuracy)






# Шаг 3. Предсказание на тестовых данных
y_pred = KNN.predict(X_test_scaled)

# Шаг 4. Оценка предсказания по метрике accuracy
accuracy = accuracy_score(y_test, y_pred)

# Шаг 4. Оценка предсказания по метрике accuracy

print("Accuracy : %.4f" % accuracy)





# Шаг 3. Предсказание на тестовых данных
y_pred = SVC.predict(X_test_scaled)

# Шаг 4. Оценка предсказания по метрике accuracy
accuracy = accuracy_score(y_test, y_pred)

# Шаг 4. Оценка предсказания по метрике accuracy

print("Accuracy : %.4f" % accuracy)



# Шаг 3. Предсказание на тестовых данных
y_pred = DTC.predict(X_test_scaled)

# Шаг 4. Оценка предсказания по метрике accuracy
accuracy = accuracy_score(y_test, y_pred)

# Шаг 4. Оценка предсказания по метрике accuracy

print("Accuracy : %.4f" % accuracy)


Accuracy : 0.8077
Accuracy : 0.8013
Accuracy : 0.7949
Accuracy : 0.8141
Accuracy : 0.7917
Accuracy : 0.7853
Accuracy : 0.8109
Accuracy : 0.7404


In [ ]:
isAlive
culture_Braavosi
culture_Reach
culture_Westermen
house_Faith of the Seven
house_House Frey
house_House Greyjoy
house_House Osgrey
house_House Tyrell
title_Archmaester
title_Lady
title_Lord
title_Septon
title_Winterfell
spouse_Alys Arryn
spouse_Amerei Frey
spouse_Craster
spouse_Kiera of Tyrosh
spouse_Rohanne Webber
spouse_Stevron Frey
spouse_Viserys I Targaryen
spouse_Walder Frey

In [251]:
data = pd.read_csv('game_of_thrones_test.csv', index_col = 'S.No')

data.columns

Index(['name', 'title', 'male', 'culture', 'dateOfBirth', 'mother', 'father',
       'heir', 'house', 'spouse', 'book1', 'book2', 'book3', 'book4', 'book5',
       'isAliveMother', 'isAliveFather', 'isAliveHeir', 'isAliveSpouse',
       'isMarried', 'isNoble', 'age', 'numDeadRelations', 'popularity'],
      dtype='object')

In [252]:
#Index(['name', 'title', 'male', 'culture', 'dateOfBirth', 'mother', 'father',
 #      'heir', 'house', 'spouse', 'book1', 'book2', 'book3', 'book4', 'book5',
  #     'isAliveMother', 'isAliveFather', 'isAliveHeir', 'isAliveSpouse',
   #    'isMarried', 'isNoble', 'age', 'numDeadRelations', 'popularity',
    #   'isAlive'],
     # dtype='object')

drop_column_list = [ 'house','culture','name','mother','father', 'heir',
       'isAliveMother', 'isAliveFather', 'isAliveHeir', 'dateOfBirth']
data.drop(columns=drop_column_list, inplace=True)

In [253]:
for column_name, column_data in data.items():
    na_percentage = round((column_data.isna().sum() / len(column_data)) * 100, 3)
    print(f'{column_name}: {na_percentage} % пропущенных данных')

title: 43.188 % пропущенных данных
male: 0.0 % пропущенных данных
spouse: 80.463 % пропущенных данных
book1: 0.0 % пропущенных данных
book2: 0.0 % пропущенных данных
book3: 0.0 % пропущенных данных
book4: 0.0 % пропущенных данных
book5: 0.0 % пропущенных данных
isAliveSpouse: 80.463 % пропущенных данных
isMarried: 0.0 % пропущенных данных
isNoble: 0.0 % пропущенных данных
age: 60.411 % пропущенных данных
numDeadRelations: 0.0 % пропущенных данных
popularity: 0.0 % пропущенных данных


In [255]:
#mean_age = data['age'][data['age'].notna()].mean()
#mean_age

# Отдельно заменим NaN на 'Rare'
data['title'] = data['title'].fillna('Rare')
##data['culture'] = data['culture'].fillna('Outliers')
##data['house'] = data['house'].fillna('Uncommon')
data['isAliveSpouse'] = data['isAliveSpouse'].fillna(1)
data['age'] = data['age'].fillna(35.29)
data['spouse']= data['spouse'].fillna('Unc')

# Проверим результат


# Находим количество встречаемых значений для каждого дома
house_counts = data['title'].value_counts()

# Определяем дома, которые встречаются один раз
rare_title = house_counts[house_counts <= 9].index

# Заменяем редкие дома (встречающиеся 1 раз) на 'Uncommon'
data['title'] = data['title'].replace(rare_title, 'Rare')




# Подсказка
#Упростите признак culture, объединив схожие названия в один.
cult = {
    'Summer Islands': ['summer islands', 'summer islander', 'summer isles'],
    'Ghiscari': ['ghiscari', 'ghiscaricari',  'ghis'],
    'Asshai': ["asshai'i", 'asshai'],
    'Lysene': ['lysene', 'lyseni'],
    'Andal': ['andal', 'andals'],
    'Braavosi': ['braavosi', 'braavos'],
    'Dornish': ['dornishmen', 'dorne', 'dornish'],
    'Myrish': ['myr', 'myrish', 'myrmen'],
    'Westermen': ['westermen', 'westerman', 'westerlands'],
    'Westerosi': ['westeros', 'westerosi'],
    'Stormlander': ['stormlands', 'stormlander'],
    'Norvoshi': ['norvos', 'norvoshi'],
    'Northmen': ['the north', 'northmen'],
    'Free Folk': ['wildling', 'first men', 'free folk'],
    'Qartheen': ['qartheen', 'qarth'],
    'Reach': ['the reach', 'reach', 'reachmen'],
}

# Переворачиваем словарь для обратного поиска (когда по значению можно найти ключ)
#reverse_cult = {alias: key for key, aliases in cult.items() for alias in aliases}

#data['culture'] = data['culture'].apply(lambda x: reverse_cult[x.lower()] if x.lower() in reverse_cult.keys() else 'Outliers' )

#sorted(data['culture'].unique())

# Находим количество встречаемых значений для каждого дома
#cult_counts = data['culture'].value_counts()

# Определяем дома, которые встречаются один раз
#rare_title = cult_counts[cult_counts <= 6].index

# Заменяем редкие дома (встречающиеся 1 раз) на 'Uncommon'
#data['culture'] = data['culture'].replace(rare_title, 'Outliers')






#house_counts = data['house'].value_counts()

# Определяем дома, которые встречаются один раз
#rare_houses = house_counts[house_counts <= 12].index

# Заменяем редкие дома (встречающиеся 1 раз) на 'Uncommon'
#data['house'] = data['house'].replace(rare_houses, 'Uncommon')

#data['house'].value_counts()



# Находим количество встречаемых значений для каждого дома
house_counts = data['spouse'].value_counts()

# Определяем дома, которые встречаются один раз
rare_title = house_counts[house_counts <= 1].index

# Заменяем редкие дома (встречающиеся 1 раз) на 'Uncommon'
data['spouse'] = data['spouse'].replace(rare_title, 'Unc')




# Шаг 1: Находим среднее значение для строк, где 'age' >= 0
mean_age = data['age'][(data['age'].notna()) & (data['age'] >= 0)].mean()

# Шаг 2: Заменяем значения, которые NaN или меньше 0, на найденное среднее
data['age'] = data['age'].apply(lambda x: mean_age if pd.isna(x) or x < 0 else x)

# Шаг 3: Разделение на категории, перезаписываем столбец 'age'
bins = [0, 15, 25, 40, 65, float('inf')]  # Границы для категорий
labels = ['1', '2', '3', '4', '5']  # Метки для категорий

# Используем pd.cut для разделения на категории и перезаписываем столбец 'age'
data['age'] = pd.cut(data['age'], bins=bins, labels=labels, right=False)

# Проверяем результат





data['isPopular'] = np.where(data['popularity'] >= 0.5, 1.0, 0.0)

drop_column_list = ['popularity']
data.drop(columns=drop_column_list, inplace=True)
data['isPopular']






data['boolDeadRelations'] = np.where(data['numDeadRelations'] > 0, 1.0, 0.0)

data[['boolDeadRelations','numDeadRelations']]
drop_column_list = ['numDeadRelations']
data.drop(columns=drop_column_list, inplace=True)
data

,title,male,spouse,book1,book2,book3,book4,book5,isAliveSpouse,isMarried,isNoble,age,isPopular,boolDeadRelations
S.No,,,,,,,,,,,,,,
1558,Rare,0,Unc,0,0,0,0,1,1.0,0,0,5,0.0,0.0
1559,Rare,0,Unc,0,1,1,1,1,1.0,0,0,3,1.0,0.0
1560,Rare,0,Unc,0,0,1,1,0,1.0,0,0,3,0.0,0.0
1561,Ser,1,Unc,1,1,1,1,1,1.0,0,1,3,0.0,0.0
1562,Rare,1,Unc,1,1,1,1,1,1.0,1,0,3,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1942,Rare,1,Unc,1,1,1,1,1,1.0,0,0,3,0.0,0.0
1943,Rare,1,Unc,0,1,0,1,1,1.0,0,0,3,0.0,0.0
1944,Rare,1,Unc,1,1,1,1,1,1.0,0,0,3,0.0,0.0


In [256]:

from sklearn.preprocessing import LabelEncoder
import pandas as pd

# Создаем копию исходных данных
df = data.copy()

# Инициализируем LabelEncoder
label_encoder = LabelEncoder()

# Для каждой категориальной переменной применим Label Encoding
categorical_columns = [ 'title', 'spouse', 'age']

# Применяем Label Encoding к каждой категориальной переменной
for column in categorical_columns:
    # Применяем Label Encoding к столбцу
    df[column] = label_encoder.fit_transform(df[column].astype(str))  # Преобразуем в строку для работы с NaN

# Проверяем результат
print(df.head())


      title  male  spouse  book1  book2  book3  book4  book5  isAliveSpouse  \
S.No                                                                          
1558      1     0       1      0      0      0      0      1            1.0   
1559      1     0       1      0      1      1      1      1            1.0   
1560      1     0       1      0      0      1      1      0            1.0   
1561      2     1       1      1      1      1      1      1            1.0   
1562      1     1       1      1      1      1      1      1            1.0   

      isMarried  isNoble  age  isPopular  boolDeadRelations  
S.No                                                         
1558          0        0    4        0.0                0.0  
1559          0        0    2        1.0                0.0  
1560          0        0    2        0.0                0.0  
1561          0        1    2        0.0                0.0  
1562          1        0    2        0.0                0.0  


In [257]:
#import pandas as pd

# Пример датафреймов (data1 и data2)
# data1 - первый датафрейм
# data2 - второй датафрейм

for column in data1.columns:
   if column not in data2.columns:
       print(column)
        # Если столбец отсутствует в data2, добавляем его и заполняем модой из data1
    #    mode_value = data1[column].mode()[0]  # Находим моду для столбца в data1
     #   data2[column] = mode_value  # Добавляем столбец в data2 и заполняем его модой

# Проверяем результат
print(data2)


title
culture
house
spouse
age
isAlive
      male  book1  book2  book3  book4  book5  isAliveSpouse  isMarried  \
S.No                                                                      
1558     0      0      0      0      0      1            1.0          0   
1559     0      0      1      1      1      1            1.0          0   
1560     0      0      0      1      1      0            1.0          0   
1561     1      1      1      1      1      1            1.0          0   
1562     1      1      1      1      1      1            1.0          1   
...    ...    ...    ...    ...    ...    ...            ...        ...   
1942     1      1      1      1      1      1            1.0          0   
1943     1      0      1      0      1      1            1.0          0   
1944     1      1      1      1      1      1            1.0          0   
1945     1      0      0      1      1      1            1.0          0   
1946     1      0      0      1      1      1            1.0 

In [258]:


X = df.values
#y = df['spouse_encoded']
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X)
# Выход pca - numpy матрица, положим ее в новую переменную со всеми фичами
X = scaler.transform(X)


# Шаг 2. обучение модели
#logistic_regression.fit(X, y)

# Шаг 3. Предсказание на тестовых данных
#y_pred = logistic_regression.predict(X_test_scaled)

In [259]:
y_pred =  SVC.predict(X)



submission = pd.read_csv("submission.csv", index_col='S.No')


submission['isAlive'] = y_pred



In [260]:
submission['isAlive']

S.No
1558    0
1559    1
1560    1
1561    1
1562    1
       ..
1942    1
1943    1
1944    1
1945    1
1946    1
Name: isAlive, Length: 389, dtype: int64

In [261]:



submission.to_csv("new_submission.csv", index=False)

In [ ]:
# ...

In [ ]:
# ...

In [ ]:
# ...

**Задание 1.8.** Проанализируйте влияние признаков на целевую переменную.

In [ ]:
# ...

**Задание 1.9.** Создайте переменные `X`, которая будет хранить только значения признаков, и `y`, которая будет хранить только значения целевой переменной.

In [ ]:
X = # ...
y = # ...

**Задание 1.10.** Разделите датасет на train и test часть при помощи функции `train_test_split`

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = # ...

## Часть 2. Обучение моделей

В данной части домашнего задания, мы хотим научиться обучать модели для задачи классификации на наших данных.

**Задание 2.1.** Импортируйте следующие модели из библиотеки `sklearn`
* LogisticRegression
* RandomForestClassifier
* AdaBoostClassifier
* GaussianProcessClassifier
* GaussianNB
* KNeighborsClassifier
* SVC
* DecisionTreeClassifier


В качестве примера, импортируем модель `LogisticRegression`

In [ ]:
from sklearn.linear_model import LogisticRegression

Импортируйте остальные модели из библиотеки `sklearn`. Чтобы понять как это сделать, воспользуйтесь официальный документацией `sklearn` $→$ [тык](https://scikit-learn.org/dev/user_guide.html). По ключевому названию модели, вы сможете найти необходимую информацию о том, как можно импортировать модель из библиотеки.

In [ ]:
from ... import AdaBoostClassifier, RandomForestClassifier
from ... import GaussianProcessClassifier
from ... import GaussianNB
from ... import KNeighborsClassifier
from ... import SVC
from ... import DecisionTreeClassifier

**Задание 2.2.** Обучите модель и сделайте предсказание на тестовой выборке

В качестве примера, обучим модель `LogisticRegression` и сделаем на ней предсказания на тестовой выборке.

In [ ]:
# Шаг 1. создание модели
logistic_regression = LogisticRegression(C=1)

# Шаг 2. обучение модели
logistic_regression.fit(X_train, y_train)

# Шаг 3. Предсказание на тестовых данных
y_pred = logistic_regression.predict(X_test)

Аналогичным образом обучите остальные модели и сделайте предсказания на тестовой выборке.

In [ ]:
# Шаг 1. создание модели


# Шаг 2. обучение модели


# Шаг 3. Предсказание на тестовых данных


## Часть 3. Оцените качество моделей

Вам необходимо познакомиться с метриками задачи классификации из sklearn. Оцените все модели и выберите лучшую по метрике качества Accuracy.

С метриками классификации вы можете ознакомиться в [Yandex ML Book](https://education.yandex.ru/handbook/ml/article/metriki-klassifikacii-i-regressii).

Для простоты в данном домашнем задании мы будем работать с самой базовой метрикой для задачи классификации - accuracy.

**Задание 3.1.** Вам необходимо посчитать метрику для всех моделей и выбрать лучшую модель.

Сначала импортируем необходимую функцию из библиотеки sklearn для подсчета accuracy.

In [ ]:
from sklearn.metrics import accuracy_score

В качестве примера, посчитаем метрику accuracy для модели `LogisticRegression`

In [ ]:
# Шаг 3. Предсказание на тестовых данных
y_pred = logistic_regression.predict(X_test)

# Шаг 4. Оценка предсказания по метрике accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy : %.4f" % accuracy)

Аналогичным образом посчитайте метрику accuracy для остальных моделей

In [ ]:
# Шаг 3. Предсказание на тестовых данных


# Шаг 4. Оценка предсказания по метрике accuracy

print("Accuracy : %.4f" % accuracy)

Выберите лучшую модель.

### Файл `submission.csv`

Вам нужно вместо значений в `submission.csv` файле в колонке `isAlive`, подставить свои предсказания и сохранить измененный файл.

In [ ]:
!gdown 1M14conWjAW2QLoyCXbHEAy8bql2f99eF

In [ ]:
submission = pd.read_csv("/content/submission.csv", index_col='S.No')

In [ ]:
submission

Как сохранить измененный Pandas DataFrame в csv файл:

In [ ]:
submission.to_csv("/content/new_submission.csv", index=False)